## Number of documents over time

In [1]:
import pandas as pd
import altair as alt

from utils import pgp_csv_paths, chart_dir

documents = pd.read_csv(pgp_csv_paths["documents"])

In [2]:
# parse initial entry as date time
documents['initial_entry'] = pd.to_datetime(documents['initial_entry'], format='mixed')


#documents['pgpid'].groupby(documents['initial_entry'].dt.to_period('Y')).sum().plot(kind='bar')
newdocs_per_year = documents['pgpid'].groupby(documents['initial_entry'].dt.to_period('Y')).count().reset_index().rename(columns={'pgpid': 'total'})
# convert time period to timestamp
newdocs_per_year["initial_entry"] = newdocs_per_year["initial_entry"].dt.to_timestamp()
newdocs_per_year.head(10)

/var/folders/mb/6qm4h4yx3yqdy2bv2sjyp4z00000gp/T/ipykernel_23739/527621908.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  newdocs_per_year = documents['pgpid'].groupby(documents['initial_entry'].dt.to_period('Y')).count().reset_index().rename(columns={'pgpid': 'total'})


,initial_entry,total
0,1986-01-01,138
1,1987-01-01,337
2,1988-01-01,362
3,1989-01-01,235
4,1990-01-01,559
5,1991-01-01,26
6,2002-01-01,1
7,2004-01-01,1263
8,2010-01-01,8
9,2014-01-01,1


In [3]:
# calculate cumulative total
newdocs_per_year['cumulative_total'] = newdocs_per_year['total'].cumsum()
newdocs_per_year.head()

,initial_entry,total,cumulative_total
0,1986-01-01,138,138
1,1987-01-01,337,475
2,1988-01-01,362,837
3,1989-01-01,235,1072
4,1990-01-01,559,1631


In [4]:
# use melt to revise so we can plot both together
newdocs_per_year_melted = pd.melt(newdocs_per_year.rename(columns={'initial_entry': 'date', 'total': 'New documents', 'cumulative_total': 'Cumulative total'}), id_vars=['date'], value_vars=['New documents', 'Cumulative total'])
newdocs_per_year_melted = newdocs_per_year_melted.rename(columns={"variable": "status", "value": "Documents"})
newdocs_per_year_melted.head()

,date,status,Documents
0,1986-01-01,New documents,138
1,1987-01-01,New documents,337
2,1988-01-01,New documents,362
3,1989-01-01,New documents,235
4,1990-01-01,New documents,559


In [5]:
# TODO rename total/cumulative total so they make sense here
docs_overtime_chart = alt.Chart(newdocs_per_year_melted).mark_area().encode(
    y=alt.Y('Documents'),
    x=alt.Y('date', title="Year"),
    color=alt.Color("status", title="").scale(scheme="tableau20"),
).properties(
#    title="Documents in PGP per year with cumulative total",
    width=800,
    height=250

).configure_legend(
    strokeColor='gray',
    fillColor='white', #EEEEEE',
    padding=15,
    cornerRadius=5,
    orient='top-left'
)
docs_overtime_chart.save(f'{chart_dir}/docs_over_time.pdf')

docs_overtime_chart

alt.Chart(...)